# 2-1. 프로젝트: 베트남 음식 분류하기

In [1]:
import os
import pathlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

손상된 이미지 파일들을 삭제

In [2]:
# training set과 test set의 모든 이미지 파일에 대해서,
# jpg image header가 포함되지 않은 (jpg의 파일 구조에 어긋나는) 파일들을 삭제해줍니다.

data_path = '/aiffel/aiffel/model-fit/data/30vnfoods/'
train_path = data_path + 'Train/'
test_path = data_path + 'Test/'

for path in [train_path, test_path]:
    classes = os.listdir(path)

    for food in classes:
        food_path = os.path.join(path, food)
        images = os.listdir(food_path)
        
        for image in images:
            with open(os.path.join(food_path, image), 'rb') as f:
                bytes = f.read()
            if bytes[:3] != b'\xff\xd8\xff':
                print(os.path.join(food_path, image))
                os.remove(os.path.join(food_path, image))

삭제되고 난 후의 훈련 데이터 수는 아래와 같이 구할 수 있습니다.

In [3]:
classes = os.listdir(train_path)
train_length = 0

for food in classes:
    food_path = os.path.join(train_path, food)
    images = os.listdir(food_path)
    
    train_length += len(images)

print('training data의 개수: '+str(train_length))

training data의 개수: 9775


## 문제1: dataloader 구현하기
: 주어진 함수들과 TensorFlow Data API를 이용해서 dataloader를 구현해야 합니다.

In [4]:
# 문제1: dataloader 구현하기

def process_path(file_path, class_names, img_shape=(224, 224)):
    '''
    file_path로 부터 class label을 만들고, 이미지를 읽는 함수
    이미지 크기를 (224, 224)로 맞춰주세요.
    '''
    label = tf.strings.split(file_path, os.path.sep)
    label = label[-2] == class_names

    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, img_shape)
    
    return img, label

def prepare_for_training(ds, batch_size=32, cache=True, shuffle_buffer_size=1000, n_repeat=1):
    '''
    TensorFlow Data API를 이용해 data batch를 만드는 함수
    '''
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.repeat(n_repeat)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds

def load_data(data_path, batch_size=32):
    '''
    데이터를 만들기 위해 필요한 함수들을 호출하고 데이터를 리턴해주는 함수
    TensorFlow Dataset 객체를 생성하고 process_path 함수로 이미지와 라벨을 묶은 다음,
    prepare_for_training 함수로 batch가 적용된 Dataset 객체를 만들어주세요.
    '''
    class_names = [cls for cls in os.listdir(data_path) if cls != '.DS_Store']
    data_path = pathlib.Path(data_path)

    list_ds = tf.data.Dataset.list_files(str(data_path/'*/*'))
    labeled_ds = list_ds.map(lambda x: process_path(x, class_names, img_shape=(224, 224)))
    ds = prepare_for_training(labeled_ds, batch_size=batch_size)


    return ds

## 문제2: 모델 구현하기
- EfficientNetB0 backbone과 Dense 레이어를 결합하여 모델을 구현해야 합니다.

- 다른 모델을 사용하여 backbone을 만들어 보아도 좋습니다.

In [5]:
# 문제2: 모델 구현하기

from tensorflow.keras.applications import EfficientNetB0

class Model(tf.keras.Model):
    '''
    EfficientNetB0을 백본으로 사용하는 모델을 구성합니다.
    Classification 문제로 접근할 것이기 때문에 맨 마지막 Dense 레이어에 
    우리가 원하는 클래스 개수만큼을 지정해주어야 합니다.
    '''
    def __init__(self, num_classes=10, freeze=False):
        super(Model, self).__init__()
        self.base_model = EfficientNetB0(include_top=False, weights='imagenet')
        if freeze:
            self.base_model.trainable = False
        self.top = tf.keras.Sequential([tf.keras.layers.GlobalAveragePooling2D(name="avg_pool"),
                                       tf.keras.layers.BatchNormalization(),
                                       tf.keras.layers.Dropout(0.5, name="top_dropout")])
        self.classifier = tf.keras.layers.Dense(num_classes, activation="softmax", name="pred")

    def call(self, inputs, training=True):
        x = self.base_model(inputs)
        x = self.top(x)
        x = self.classifier(x)
        return x
    
if __name__ == '__main__':
    model = Model(num_classes=10, freeze=True)
    model.build(input_shape=(None, 224, 224, 3))
    print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
sequential (Sequential)      (None, 1280)              5120      
_________________________________________________________________
pred (Dense)                 multiple                  12810     
Total params: 4,067,501
Trainable params: 15,370
Non-trainable params: 4,052,131
_________________________________________________________________
None


## 문제3: custom trainer 구현하기
- TensorFlow GradientTape를 이용해서 custom trainer 클래스를 구현해야 합니다.
- 학습이 진행되면서 training accuracy가 점차 증가할 수 있도록 만들어 보세요.

In [6]:
# 문제3: custom trainer 구현하기

class Trainer:
    def __init__(self, model, epochs, batch, loss_fn, optimizer):
        self.model = model
        self.epochs = epochs
        self.batch = batch
        self.loss_fn = loss_fn
        self.optimizer = optimizer
    def train(self, train_dataset, train_metric):
        for epoch in range(self.epochs):
            print("\nStart of epoch %d" % (epoch+1,))
            # [[YOUR CODE]]
            for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
                with tf.GradientTape() as tape:
                    logits = model(x_batch_train, training=True)
                    loss_value = self.loss_fn(y_batch_train, logits)
                grads = tape.gradient(loss_value, model.trainable_weights)
                self.optimizer.apply_gradients(zip(grads, model.trainable_weights))
                # train metric 업데이트
                train_metric.update_state(y_batch_train, logits)
                # 5 배치마다 로깅
                if step % 5 == 0:
                    print(
                        "Training loss (for one batch) at step %d: %.4f"
                        % (step, float(loss_value))
                    )
                    print("Seen so far: %d samples" % ((step + 1) * self.batch))
                    print(train_metric.result().numpy())            
            # 마지막 epoch 학습이 끝나면 train 결과를 보여줌
            train_acc = train_acc_metric.result()
            print("Training acc over epoch: %.4f" % (float(train_acc),))

In [7]:
# 모델 학습 코드

train_path = "/aiffel/aiffel/model-fit/data/30vnfoods/Train"

epoch = 5
batch = 32

model = Model(num_classes=10)
dataset = load_data(data_path=train_path, batch_size=batch)
loss_function = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
trainer = Trainer(model=model,
                epochs=epoch,
                batch=batch,
                #ds_length=train_length,
                loss_fn=loss_function,
                optimizer=optimizer)

trainer.train(train_dataset=dataset,
            train_metric=train_acc_metric)


Start of epoch 1
Training loss (for one batch) at step 0: 4.0263
Seen so far: 32 samples
0.0625
Training loss (for one batch) at step 5: 2.6789
Seen so far: 192 samples
0.14583333
Training loss (for one batch) at step 10: 2.3200
Seen so far: 352 samples
0.23579545
Training loss (for one batch) at step 15: 1.4417
Seen so far: 512 samples
0.29492188
Training loss (for one batch) at step 20: 1.5462
Seen so far: 672 samples
0.36011904
Training loss (for one batch) at step 25: 0.8715
Seen so far: 832 samples
0.40144232
Training loss (for one batch) at step 30: 1.3830
Seen so far: 992 samples
0.4375
Training loss (for one batch) at step 35: 1.0832
Seen so far: 1152 samples
0.47743055
Training loss (for one batch) at step 40: 1.4412
Seen so far: 1312 samples
0.49618903
Training loss (for one batch) at step 45: 0.7839
Seen so far: 1472 samples
0.515625
Training loss (for one batch) at step 50: 1.2562
Seen so far: 1632 samples
0.5275735
Training loss (for one batch) at step 55: 0.9102
Seen so 

### ㄴ 결과

TensorFlow GradientTape를 이용해서 custom trainer 클래스를 구현하였고,

학습과정에서 training accuracy가 점차 증가하는 것을 확인하였다.

### Test 결과 확인

In [8]:
# 모델 테스트 코드

test_ds = load_data(data_path=test_path)

for step_train, (x_batch_train, y_batch_train) in enumerate(test_ds.take(10)):
    prediction = model(x_batch_train)
    print("{}/{}".format(np.array(tf.equal(tf.argmax(y_batch_train, axis=1), tf.argmax(prediction, axis=1))).sum(), tf.argmax(y_batch_train, axis=1).shape[0]))

29/32
29/32
27/32
30/32
30/32
28/32
30/32
26/32
30/32
29/32


ㄴ 평균적으로 약 90%에 달하는 예측성능을 확인하였다.

## 회고
- 배운 점
1. low-level 단계에서부터 신경망을 학습하는 과정을 구현하면서 지금까지 이해가 부족했던 코드에 대한 이해도를 보충할 수 있었다.
2. tf.GradientTape() 함수를 통해 gradient를 구하고, optimizer.apply_gradients() 함수를 통하여 가중치를 갱신하는 과정을 구현할 수 있었다.
3. load_data() 함수와 prepare_for_training() 함수를 직접 코드로 구현하면서 배치단위로 데이터를 가져오는 과정, 셔플링, 학습전 배치단위로의 샘플링, prefetch와 캐싱을 통한 속도 향상, 메모리 문제해결 과정을 이해할 수 있었다. 이전 노드에서 이와 같은 방법으로 코드가 구현된 것을 많이 봤지만, 이번 프로젝트를 수행하면서 부족한 이해를 채워 넣을 수 있었다.

- 문제해결
1. 학습과정에서 훈련이 무한정 진행되는 문제가 발생하였다. prepare_for_training 함수 안에 repeat() 메서드의 인자를 추가해야 정해진 만큼 반복을 진행하는 것을 파악하였다.
2. 이번 태스크에서는 Trainer() 함수 안에 ds_length 인자가 필요하지 않으므로 삭제하였다. 